In [ ]:
import pandas as pd
import numpy as np

def read_alternatives_criteria_matrix(file_path):
    """
    Reads the Excel file, extracts the desired columns and rows, and returns:
      - The full DataFrame with alternative names.
      - A NumPy matrix of the numeric criteria values.

    The expected columns are:
        'Alternatives/Criteria', 'C1(Min)', 'C2(Max)', 'C3(Max)', 'C4(Max)'
    Only rows where 'Alternatives/Criteria' starts with 'A' (e.g., A1, A2, ...) are kept.
    """
    try:
        # Read only the first five columns (A:E) from the Excel file.
        df = pd.read_excel(file_path, usecols="A:E", header=0)

        # Filter rows where 'Alternatives/Criteria' starts with 'A'
        df = df[df["Alternatives/Criteria"].astype(str).str.startswith("A")]

        # Reset index for neatness (optional)
        df.reset_index(drop=True, inplace=True)

        # Extract the numeric part (columns 2 to 5) as a NumPy matrix.
        numeric_matrix = df.iloc[:, 1:].to_numpy()

        return df, numeric_matrix
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return None, None
    except Exception as e:
        print("An error occurred while reading the Excel file:", e)
        return None, None

def main():
    # Update the file path to your actual Excel file.
    file_path = 'excel.xlsx'  # Replace with your file path if needed.

    # Read the Excel file and extract the matrix.
    df, matrix = read_alternatives_criteria_matrix(file_path)

    if df is not None and matrix is not None:
        # Display the DataFrame for visual confirmation.
        print("Excel Alternatives/Criteria Matrix:")
        print(df.to_string(index=False))

        # Display the extracted numeric matrix.
        print("\nExtracted Numeric Matrix (for later use):")
        print(matrix)
    else:
        print("Failed to load the matrix.")

if __name__ == "__main__":
    main()

Excel Alternatives/Criteria Matrix:
Alternatives/Criteria  C1(Min)  C2(Max)  C3(Max)  C4(Max)
                   A1    250.0     16.0     12.0      5.0
                   A2    200.0     16.0      8.0      3.0
                   A3    300.0     32.0     16.0      4.0
                   A4    275.0     32.0      8.0      4.0
                   A5    225.0     16.0     16.0      2.0

Extracted Numeric Matrix (for later use):
[[250.  16.  12.   5.]
 [200.  16.   8.   3.]
 [300.  32.  16.   4.]
 [275.  32.   8.   4.]
 [225.  16.  16.   2.]]


In [ ]:
import numpy as np
n = int(input("Enter the number of alternatives: "))
m = int(input("Enter the number of criteria: "))
delta = np.random.randint(1, 101, size=(n, m))
print(delta)

Enter the number of alternatives: 4
Enter the number of criteria: 3
[[ 1 41 39]
 [67 21 90]
 [41 89 62]
 [35 39 53]]


In [ ]:
import numpy as np

# Example numeric matrix (from Excel) for demonstration
matrix = np.array([
    [250., 16., 12.,  5.],
    [200., 16.,  8.,  3.],
    [300., 32., 16.,  4.],
    [275., 32.,  8.,  4.],
    [225., 16., 16.,  2.]
])

def normalize_matrix_columnwise(mat):
    """
    Normalizes the matrix so that each column sums to 1.
    p_ij = x_ij / sum(x_ij) for each column j.
    """
    col_sums = np.sum(mat, axis=0)    # Sum over rows => one sum per column
    normalized = mat / col_sums       # Element-wise division
    return normalized

# Perform the column-wise normalization
normalized = normalize_matrix_columnwise(matrix)

# Print the result
print("Normalized Matrix:")
print(np.round(normalized, 4))  # Rounded for readability

# Verify that each column sums to ~1
print("\nColumn sums:")
print(np.sum(normalized, axis=0))


Normalized Matrix:
[[0.2    0.1429 0.2    0.2778]
 [0.16   0.1429 0.1333 0.1667]
 [0.24   0.2857 0.2667 0.2222]
 [0.22   0.2857 0.1333 0.2222]
 [0.18   0.1429 0.2667 0.1111]]

Column sums:
[1. 1. 1. 1.]


In [ ]:
import numpy as np

def normalize_matrix_columnwise(mat):
    """
    Normalizes the matrix so that each column sums to 1.
    p_ij = x_ij / sum(x_ij) for each column j.
    """
    col_sums = np.sum(mat, axis=0)  # Sum over rows => one sum per column
    normalized = mat / col_sums     # Element-wise division
    return normalized

# Get user inputs
n = int(input("Enter the number of alternatives: "))
m = int(input("Enter the number of criteria: "))

# Generate random matrix
delta = np.random.randint(1, 101, size=(n, m))
print("Random Matrix (delta):")
print(delta)

# Normalize the random matrix column-wise
delta_normalized = normalize_matrix_columnwise(delta)

print("\nNormalized Delta Matrix:")
print(delta_normalized)

# (Optional) Verify that each column sums to ~1
print("\nColumn sums of Normalized Delta Matrix:")
print(np.sum(delta_normalized, axis=0))

Enter the number of alternatives: 4
Enter the number of criteria: 3
Random Matrix (delta):
[[ 64   9   5]
 [100  41  92]
 [ 96  77  43]
 [ 36  61  31]]

Normalized Delta Matrix:
[[0.21621622 0.04787234 0.02923977]
 [0.33783784 0.21808511 0.5380117 ]
 [0.32432432 0.40957447 0.25146199]
 [0.12162162 0.32446809 0.18128655]]

Column sums of Normalized Delta Matrix:
[1. 1. 1.]


In [ ]:
import numpy as np

def compute_entropy(matrix):
    """
    Computes the entropy values E_j for each criterion (column) in the normalized matrix.
    """
    n, m = matrix.shape  # Get number of alternatives (rows) and criteria (columns)
    k = 1 / np.log(n)  # Compute k = 1 / ln(n)

    # Compute p_ij * ln(p_ij), avoiding log(0) by replacing zeros with a small value
    epsilon = 1e-10  # Small constant to avoid log(0)
    pij_ln_pij = np.where(matrix > 0, matrix * np.log(matrix + epsilon), 0)

    # Sum over rows to get column-wise sums
    column_sums = np.sum(pij_ln_pij, axis=0)

    # Compute entropy values
    entropy_values = -k * column_sums

    return entropy_values

# Example matrix (normalized)
p_ij = np.array([
    [0.2,    0.1429, 0.2,    0.2778],
    [0.16,   0.1429, 0.1333, 0.1667],
    [0.24,   0.2857, 0.2667, 0.2222],
    [0.22,   0.2857, 0.1333, 0.2222],
    [0.18,   0.1429, 0.2667, 0.1111]
])

# Compute entropy
entropy_values = compute_entropy(p_ij)
print("Entropy values (Ej):", entropy_values)

Entropy values (Ej): [0.99375089 0.96303213 0.97182082 0.973665  ]


In [ ]:
import numpy as np

def normalize_matrix_columnwise(mat):
    """
    Normalizes the matrix so that each column sums to 1.
    p_ij = x_ij / sum(x_ij) for each column j.
    """
    col_sums = np.sum(mat, axis=0)  # Sum over rows => one sum per column
    normalized = mat / col_sums     # Element-wise division
    return normalized

def compute_entropy(matrix):
    """
    Computes the entropy values E_j for each criterion (column) in the normalized matrix.
    Displays each step of the computation.
    """
    n, m = matrix.shape  # Get number of alternatives (rows) and criteria (columns)
    k = 1 / np.log(n)  # Compute k = 1 / ln(n)

    # Step 1: Compute p_ij * ln(p_ij), avoiding log(0) by replacing zeros with a small value
    epsilon = 1e-10  # Small constant to avoid log(0)
    pij_ln_pij = np.where(matrix > 0, matrix * np.log(matrix + epsilon), 0)

    # Step 2: Sum over rows to get column-wise sums
    column_sums = np.sum(pij_ln_pij, axis=0)

    # Step 3: Compute entropy values
    entropy_values = -k * column_sums

    # Display results at each step
    print("\nStep 2: p_ij * ln(p_ij) Matrix:")
    print(pij_ln_pij)

    print("\nStep 3: Column-wise Sum of p_ij * ln(p_ij):")
    print(column_sums)

    print("\nStep 4: Computed k Value (1/ln(n)):")
    print(k)

    print("\nFinal Step: Entropy Values (Ej):")
    print(entropy_values)

    return entropy_values

# Get user inputs
n = int(input("Enter the number of alternatives: "))
m = int(input("Enter the number of criteria: "))

# Step 1: Generate a random decision matrix
random_matrix = np.random.randint(1, 101, size=(n, m))
print("\nStep 1: Random Decision Matrix (X_ij):")
print(random_matrix)

# Step 2: Normalize the matrix
normalized_matrix = normalize_matrix_columnwise(random_matrix)
print("\nStep 2: Normalized Matrix (p_ij):")
print(normalized_matrix)

# Step 3: Compute entropy
entropy_values = compute_entropy(normalized_matrix)


Enter the number of alternatives: 5
Enter the number of criteria: 4

Step 1: Random Decision Matrix (X_ij):
[[14 89  9 41]
 [32 24 29 37]
 [20 85 64 27]
 [26 76 25 29]
 [95 86 87 21]]

Step 2: Normalized Matrix (p_ij):
[[0.07486631 0.24722222 0.04205607 0.26451613]
 [0.17112299 0.06666667 0.13551402 0.23870968]
 [0.10695187 0.23611111 0.29906542 0.17419355]
 [0.13903743 0.21111111 0.11682243 0.18709677]
 [0.50802139 0.23888889 0.40654206 0.13548387]]

Step 2: p_ij * ln(p_ij) Matrix:
[[-0.19405732 -0.34548506 -0.13326525 -0.35176758]
 [-0.30209587 -0.18053668 -0.27084918 -0.34195333]
 [-0.23907768 -0.34081524 -0.36099976 -0.3044186 ]
 [-0.27432253 -0.32835603 -0.25082946 -0.31359838]
 [-0.3440482  -0.34203078 -0.36591545 -0.27081907]]

Step 3: Column-wise Sum of p_ij * ln(p_ij):
[-1.3536016  -1.53722379 -1.3818591  -1.58255697]

Step 4: Computed k Value (1/ln(n)):
0.6213349345596119

Final Step: Entropy Values (Ej):
[0.84103996 0.95513084 0.85859733 0.98329793]


In [ ]:
import numpy as np

def compute_entropy(matrix):
    """
    Computes the entropy values E_j for each criterion (column) in the normalized matrix.
    """
    n, m = matrix.shape  # Get number of alternatives (rows) and criteria (columns)
    k = 1 / np.log(n)  # Compute k = 1 / ln(n)

    # Compute p_ij * ln(p_ij), avoiding log(0) by replacing zeros with a small value
    epsilon = 1e-10  # Small constant to avoid log(0)
    pij_ln_pij = np.where(matrix > 0, matrix * np.log(matrix + epsilon), 0)

    # Sum over rows to get column-wise sums
    column_sums = np.sum(pij_ln_pij, axis=0)

    # Compute entropy values
    entropy_values = -k * column_sums

    return entropy_values

def compute_weights(entropy_values):
    """
    Computes the weights w_j using the formula:
    w_j = (1 - E_j) / sum(1 - E_j)
    """
    one_minus_ej = 1 - entropy_values  # Calculate (1 - E_j) for each column
    sum_one_minus_ej = np.sum(one_minus_ej)  # Sum of all (1 - E_j)
    weights = one_minus_ej / sum_one_minus_ej  # Compute w_j

    return weights

# Predefined normalized matrix (p_ij)
p_ij = np.array([
    [0.2,    0.1429, 0.2,    0.2778],
    [0.16,   0.1429, 0.1333, 0.1667],
    [0.24,   0.2857, 0.2667, 0.2222],
    [0.22,   0.2857, 0.1333, 0.2222],
    [0.18,   0.1429, 0.2667, 0.1111]
])

# Compute entropy
entropy_values = compute_entropy(p_ij)
print("Step 3: Entropy values (Ej):", entropy_values)

# Compute weights
weights = compute_weights(entropy_values)
print("\nStep 4: Weights (Wj):", weights)

Step 3: Entropy values (Ej): [0.99375089 0.96303213 0.97182082 0.973665  ]

Step 4: Weights (Wj): [0.06394183 0.37826086 0.28833359 0.26946372]


In [ ]:
import numpy as np

def normalize_matrix_columnwise(mat):
    """
    Normalizes the matrix so that each column sums to 1.
    p_ij = x_ij / sum(x_ij) for each column j.
    """
    col_sums = np.sum(mat, axis=0)  # Sum over rows => one sum per column
    normalized = mat / col_sums     # Element-wise division
    return normalized

def compute_entropy(matrix):
    """
    Computes the entropy values E_j for each criterion (column) in the normalized matrix.
    Displays each step of the computation.
    """
    n, m = matrix.shape  # Get number of alternatives (rows) and criteria (columns)
    k = 1 / np.log(n)  # Compute k = 1 / ln(n)

    # Step 1: Compute p_ij * ln(p_ij), avoiding log(0) by replacing zeros with a small value
    epsilon = 1e-10  # Small constant to avoid log(0)
    pij_ln_pij = np.where(matrix > 0, matrix * np.log(matrix + epsilon), 0)

    # Step 2: Sum over rows to get column-wise sums
    column_sums = np.sum(pij_ln_pij, axis=0)

    # Step 3: Compute entropy values
    entropy_values = -k * column_sums

    # Display results at each step
    print("\nStep 2: p_ij * ln(p_ij) Matrix:")
    print(pij_ln_pij)

    print("\nStep 3: Column-wise Sum of p_ij * ln(p_ij):")
    print(column_sums)

    print("\nStep 4: Computed k Value (1/ln(n)):")
    print(k)

    print("\nFinal Step 5: Entropy Values (Ej):")
    print(entropy_values)

    return entropy_values

def compute_weights(entropy_values):
    """
    Computes the weights w_j using the formula:
    w_j = (1 - E_j) / sum(1 - E_j)
    """
    one_minus_ej = 1 - entropy_values  # Calculate (1 - E_j) for each column
    sum_one_minus_ej = np.sum(one_minus_ej)  # Sum of all (1 - E_j)
    weights = one_minus_ej / sum_one_minus_ej  # Compute w_j

    # Display intermediate steps
    print("\nStep 6: 1 - E_j Values:")
    print(one_minus_ej)

    print("\nStep 7: Sum of (1 - E_j):")
    print(sum_one_minus_ej)

    print("\nFinal Step 8: Computed Weights (W_j):")
    print(weights)

    return weights

# Get user inputs
n = int(input("Enter the number of alternatives: "))
m = int(input("Enter the number of criteria: "))

# Step 1: Generate a random decision matrix
random_matrix = np.random.randint(1, 101, size=(n, m))
print("\nStep 1: Random Decision Matrix (X_ij):")
print(random_matrix)

# Step 2: Normalize the matrix
normalized_matrix = normalize_matrix_columnwise(random_matrix)
print("\nStep 2: Normalized Matrix (p_ij):")
print(normalized_matrix)

# Step 3: Compute entropy
entropy_values = compute_entropy(normalized_matrix)

# Step 4: Compute weights
weights = compute_weights(entropy_values)

Enter the number of alternatives: 5
Enter the number of criteria: 4

Step 1: Random Decision Matrix (X_ij):
[[ 34  57  86 100]
 [ 27  15 100  98]
 [ 55  86  54  16]
 [ 59  22  79  65]
 [ 18  50  52  40]]

Step 2: Normalized Matrix (p_ij):
[[0.1761658  0.24782609 0.23180593 0.31347962]
 [0.13989637 0.06521739 0.26954178 0.30721003]
 [0.28497409 0.37391304 0.14555256 0.05015674]
 [0.30569948 0.09565217 0.21293801 0.20376176]
 [0.09326425 0.2173913  0.14016173 0.12539185]]

Step 2: p_ij * ln(p_ij) Matrix:
[[-0.30588191 -0.34572434 -0.3388666  -0.36364292]
 [-0.27515565 -0.17804538 -0.35337786 -0.36257654]
 [-0.35774422 -0.36783023 -0.28051152 -0.15009918]
 [-0.36230058 -0.22449918 -0.32936276 -0.32414498]
 [-0.2212525  -0.33175137 -0.27541195 -0.26035256]]

Step 3: Column-wise Sum of p_ij * ln(p_ij):
[-1.52233486 -1.4478505  -1.57753069 -1.46081618]

Step 4: Computed k Value (1/ln(n)):
0.6213349345596119

Final Step 5: Entropy Values (Ej):
[0.94587983 0.89960009 0.98017493 0.90765612]

St

In [ ]:
import numpy as np

def generate_criteria_type_matrix(m, n):
    """
    Generates a random 0-1 matrix indicating whether each criterion is maximization (1) or minimization (0).
    """
    return np.random.randint(0, 2, size=(1, m))  # 1 row, m columns

def normalize_matrix(matrix, criteria_types):
    """
    Normalizes the decision matrix based on whether the criterion is maximized or minimized.
    - If Maximize (1): Normalize by dividing by the max value of the column.
    - If Minimize (0): Normalize by taking the min value of the column divided by the number.
    """
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop through each column
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        else:  # Minimization
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]

    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """
    Computes WSM scores by summing the weighted normalized values row-wise.
    """
    weighted_matrix = normalized_matrix * weights  # Element-wise multiplication
    wsm_scores = np.sum(weighted_matrix, axis=1)  # Sum over rows
    return wsm_scores

# Predefined decision matrix (delta)

delta = np.array([
    [250., 16., 12.,  5.],
    [200., 16.,  8.,  3.],
    [300., 32., 16.,  4.],
    [275., 32.,  8.,  4.],
    [225., 16., 16.,  2.]
])

# Predefined criteria types (C1 Min, C2 Max, C3 Max, C4 Max)
criteria_types_predefined = np.array([[0, 1, 1, 1]])

# Generate a random 0-1 matrix for another scenario
criteria_types_random = generate_criteria_type_matrix(delta.shape[1], delta.shape[0])

print("Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):")
print(criteria_types_random)

# Normalize matrices
normalized_predefined = normalize_matrix(delta, criteria_types_predefined)
normalized_random = normalize_matrix(delta, criteria_types_random)

print("\nNormalized Predefined Matrix:")
print(normalized_predefined)


# Use previously computed weights
weights = np.array([0.2485, 0.2422, 0.2499, 0.2594])  # From entropy method

# Compute WSM scores
wsm_scores_predefined = compute_wsm_score(normalized_predefined, weights)


print("\nWSM Scores for Predefined Matrix:")
print(wsm_scores_predefined)


Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):
[[1 0 0 1]]

Normalized Predefined Matrix:
[[0.8        0.5        0.75       1.        ]
 [1.         0.5        0.5        0.6       ]
 [0.66666667 1.         1.         0.8       ]
 [0.72727273 1.         0.5        0.8       ]
 [0.88888889 0.5        1.         0.4       ]]

WSM Scores for Predefined Matrix:
[0.766725   0.65019    0.86528667 0.75539727 0.69564889]


In [ ]:
import numpy as np

def generate_criteria_type_matrix(m):
    """
    Generates a random 0-1 matrix indicating whether each criterion is maximization (1) or minimization (0).
    """
    return np.random.randint(0, 2, size=(1, m))  # 1 row, m columns

def normalize_matrix(matrix, criteria_types):
    """
    Normalizes the decision matrix based on whether the criterion is maximized or minimized.
    - If Maximize (1): Normalize by dividing by the max value of the column.
    - If Minimize (0): Normalize by taking the min value of the column divided by the number.
    """
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop through each column
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        else:  # Minimization
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]

    return normalized_matrix

# Function to interact with the user for matrix dimensions
def user_input():
    n = int(input("Enter the number of rows (n): "))  # Number of alternatives
    m = int(input("Enter the number of columns (m): "))  # Number of criteria

    # Generate a random decision matrix (n rows, m columns)
    matrix = np.random.randint(1, 101, size=(n, m))  # Random values between 1 and 100

    # Generate a random 0-1 matrix indicating max/min for each column
    criteria_types = generate_criteria_type_matrix(m)

    print("\nDecision Matrix:")
    print(matrix)

    print("\nCriteria Type Matrix for Random Scenario (0 = Min, 1 = Max):")
    print(criteria_types)

    # Normalize the matrix
    normalized_matrix = normalize_matrix(matrix, criteria_types)

    print("\nNormalized Matrix:")
    print(normalized_matrix)

    return matrix, criteria_types, normalized_matrix

# Get user input
matrix, criteria_types, normalized_matrix = user_input()

# Use previously computed weights (if any for WSM calculation later)
weights = np.random.rand(matrix.shape[1])  # Random weights for each criterion

# Compute WSM scores (optional for future)
wsm_scores = compute_wsm_score(normalized_matrix, weights)

print("\nWSM Scores (optional, if weights are provided):")
print(wsm_scores)

Enter the number of rows (n): 5
Enter the number of columns (m): 6

Decision Matrix:
[[ 92  46  36   5  10   2]
 [ 17  39  12  98  10  58]
 [ 86  21  29  91   5  31]
 [ 33  58 100  38  73  28]
 [ 54  87   7  40  62  91]]

Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):
[[1 1 0 0 0 0]]

Normalized Matrix:
[[1.         0.52873563 0.19444444 1.         0.5        1.        ]
 [0.18478261 0.44827586 0.58333333 0.05102041 0.5        0.03448276]
 [0.93478261 0.24137931 0.24137931 0.05494505 1.         0.06451613]
 [0.35869565 0.66666667 0.07       0.13157895 0.06849315 0.07142857]
 [0.58695652 1.         1.         0.125      0.08064516 0.02197802]]

WSM Scores (optional, if weights are provided):
[2.78548476 1.55206806 1.95172771 1.05645589 2.28891108]


In [ ]:
import numpy as np

def generate_criteria_type_matrix(m):
    """
    Generates a random 0-1 matrix indicating whether each criterion is maximization (1) or minimization (0).
    """
    return np.random.randint(0, 2, size=(1, m))  # 1 row, m columns

def normalize_matrix(matrix, criteria_types):
    """
    Normalizes the decision matrix based on whether the criterion is maximized or minimized.
    - If Maximize (1): Normalize by dividing by the max value of the column.
    - If Minimize (0): Normalize by taking the min value of the column divided by the number.
    """
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop through each column
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        else:  # Minimization
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]

    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """
    Computes WSM scores by summing the weighted normalized values row-wise.
    """
    weighted_matrix = normalized_matrix * weights  # Element-wise multiplication
    wsm_scores = np.sum(weighted_matrix, axis=1)  # Sum over rows
    return wsm_scores

def compute_wpm_score(normalized_matrix, weights):
    """
    Computes WPM scores by multiplying the normalized values raised to the power of the weights.
    """
    wpm_scores = np.prod(normalized_matrix ** weights, axis=1)  # Multiply each row, with exponentiation by weights
    return wpm_scores

# Function to interact with the user for matrix dimensions
def user_input():
    n = int(input("Enter the number of rows (n): "))  # Number of alternatives
    m = int(input("Enter the number of columns (m): "))  # Number of criteria

    # Generate a random decision matrix (n rows, m columns)
    matrix = np.random.randint(1, 101, size=(n, m))  # Random values between 1 and 100

    # Generate a random 0-1 matrix indicating max/min for each column
    criteria_types = generate_criteria_type_matrix(m)

    print("\nDecision Matrix:")
    print(matrix)

    print("\nCriteria Type Matrix for Random Scenario (0 = Min, 1 = Max):")
    print(criteria_types)

    # Normalize the matrix
    normalized_matrix = normalize_matrix(matrix, criteria_types)

    print("\nNormalized Matrix:")
    print(normalized_matrix)

    return matrix, criteria_types, normalized_matrix

# Get user input
matrix, criteria_types, normalized_matrix = user_input()

# Use previously computed weights (if any for WSM and WPM calculation later)
weights = np.random.rand(matrix.shape[1])  # Random weights for each criterion

# Compute WSM scores
wsm_scores = compute_wsm_score(normalized_matrix, weights)

# Compute WPM scores
wpm_scores = compute_wpm_score(normalized_matrix, weights)

# Display the results
print("\nWSM Scores:")
print(wsm_scores)

print("\nWPM Scores:")
print(wpm_scores)

Enter the number of rows (n): 5
Enter the number of columns (m): 4

Decision Matrix:
[[83 66 33 76]
 [11 94 62  9]
 [24 40  1 79]
 [37 97  4 83]
 [89 94 47 81]]

Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):
[[0 0 1 0]]

Normalized Matrix:
[[0.13253012 0.60606061 0.53225806 0.11842105]
 [1.         0.42553191 1.         1.        ]
 [0.45833333 1.         0.01612903 0.11392405]
 [0.2972973  0.41237113 0.06451613 0.10843373]
 [0.12359551 0.42553191 0.75806452 0.11111111]]

WSM Scores:
[0.64608259 1.60491248 0.77565215 0.41774949 0.62071877]

WPM Scores:
[0.0722819  0.62750109 0.03903665 0.03336218 0.06331093]


In [ ]:
import numpy as np

def generate_criteria_type_matrix(m, n):
    """
    Generates a random 0-1 matrix indicating whether each criterion is maximization (1) or minimization (0).
    """
    return np.random.randint(0, 2, size=(1, m))  # 1 row, m columns

def normalize_matrix(matrix, criteria_types):
    """
    Normalizes the decision matrix based on whether the criterion is maximized or minimized.
    - If Maximize (1): Normalize by dividing by the max value of the column.
    - If Minimize (0): Normalize by taking the min value of the column divided by the number.
    """
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop through each column
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        else:  # Minimization
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]

    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """
    Computes WSM scores by summing the weighted normalized values row-wise.
    """
    weighted_matrix = normalized_matrix * weights  # Element-wise multiplication
    wsm_scores = np.sum(weighted_matrix, axis=1)  # Sum over rows
    return wsm_scores

def compute_wpm_score(normalized_matrix, weights):
    """
    Computes WPM scores by multiplying the normalized values raised to the power of the weights.
    """
    wpm_scores = np.prod(normalized_matrix ** weights, axis=1)  # Multiply each row, with exponentiation by weights
    return wpm_scores

# Predefined decision matrix (delta)

delta = np.array([
    [250., 16., 12.,  5.],
    [200., 16.,  8.,  3.],
    [300., 32., 16.,  4.],
    [275., 32.,  8.,  4.],
    [225., 16., 16.,  2.]
])

# Predefined criteria types (C1 Min, C2 Max, C3 Max, C4 Max)
criteria_types_predefined = np.array([[0, 1, 1, 1]])

# Generate a random 0-1 matrix for another scenario
criteria_types_random = generate_criteria_type_matrix(delta.shape[1], delta.shape[0])

print("Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):")
print(criteria_types_random)

# Normalize matrices
normalized_predefined = normalize_matrix(delta, criteria_types_predefined)
normalized_random = normalize_matrix(delta, criteria_types_random)

print("\nNormalized Predefined Matrix:")
print(normalized_predefined)

# Use previously computed weights
weights = np.array([0.2485, 0.2422, 0.2499, 0.2594])  # From entropy method

# Compute WSM scores
wsm_scores_predefined = compute_wsm_score(normalized_predefined, weights)

# Compute WPM scores
wpm_scores_predefined = compute_wpm_score(normalized_predefined, weights)

print("\nWSM Scores for Predefined Matrix:")
print(wsm_scores_predefined)

print("\nWPM Scores for Predefined Matrix:")
print(wpm_scores_predefined)

Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):
[[0 0 0 0]]

Normalized Predefined Matrix:
[[0.8        0.5        0.75       1.        ]
 [1.         0.5        0.5        0.6       ]
 [0.66666667 1.         1.         0.8       ]
 [0.72727273 1.         0.5        0.8       ]
 [0.88888889 0.5        1.         0.4       ]]

WSM Scores for Predefined Matrix:
[0.766725   0.65019    0.86528667 0.75539727 0.69564889]

WPM Scores for Predefined Matrix:
[0.74436545 0.62275263 0.8533022  0.73327339 0.64737138]


In [ ]:
import numpy as np

def generate_criteria_type_matrix(m, n):
    """
    Generates a random 0-1 matrix indicating whether each criterion is maximization (1) or minimization (0).
    """
    return np.random.randint(0, 2, size=(1, m))  # 1 row, m columns

def normalize_matrix(matrix, criteria_types):
    """
    Normalizes the decision matrix based on whether the criterion is maximized or minimized.
    - If Maximize (1): Normalize by dividing by the max value of the column.
    - If Minimize (0): Normalize by taking the min value of the column divided by the number.
    """
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop through each column
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        else:  # Minimization
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]

    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """
    Computes WSM scores by summing the weighted normalized values row-wise.
    """
    weighted_matrix = normalized_matrix * weights  # Element-wise multiplication
    wsm_scores = np.sum(weighted_matrix, axis=1)  # Sum over rows
    return wsm_scores

def compute_wpm_score(normalized_matrix, weights):
    """
    Computes WPM scores by multiplying the normalized values raised to the power of the weights.
    """
    wpm_scores = np.prod(normalized_matrix ** weights, axis=1)  # Multiply each row, with exponentiation by weights
    return wpm_scores

def compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5):
    """
    Computes WASSPASS score as a combination of WSM and WPM.
    """
    wasspass_scores = lambda_value * wsm_scores + (1 - lambda_value) * wpm_scores
    return wasspass_scores

# Predefined decision matrix (delta)
delta = np.array([
    [250., 16., 12.,  5.],
    [200., 16.,  8.,  3.],
    [300., 32., 16.,  4.],
    [275., 32.,  8.,  4.],
    [225., 16., 16.,  2.]
])

# Predefined criteria types (C1 Min, C2 Max, C3 Max, C4 Max)
criteria_types_predefined = np.array([[0, 1, 1, 1]])

# Normalize the predefined matrix
normalized_predefined = normalize_matrix(delta, criteria_types_predefined)

# Use previously computed weights (entropy method example)
weights = np.array([0.2485, 0.2422, 0.2499, 0.2594])  # From entropy method

# Compute WSM scores
wsm_scores_predefined = compute_wsm_score(normalized_predefined, weights)

# Compute WPM scores
wpm_scores_predefined = compute_wpm_score(normalized_predefined, weights)

# Compute WASSPASS scores using lambda = 0.5
wasspass_scores = compute_wasspass_score(wsm_scores_predefined, wpm_scores_predefined, lambda_value=0.5)

# Display the results
print("\nWSM Scores for Predefined Matrix:")
print(wsm_scores_predefined)

print("\nWPM Scores for Predefined Matrix:")
print(wpm_scores_predefined)

print("\nWASSPASS Scores (Combination of WSM and WPM with λ = 0.5):")
print(wasspass_scores)



WSM Scores for Predefined Matrix:
[0.766725   0.65019    0.86528667 0.75539727 0.69564889]

WPM Scores for Predefined Matrix:
[0.74436545 0.62275263 0.8533022  0.73327339 0.64737138]

WASSPASS Scores (Combination of WSM and WPM with λ = 0.5):
[0.75554522 0.63647132 0.85929443 0.74433533 0.67151013]


In [ ]:
import numpy as np

def generate_criteria_type_matrix(m):
    """
    Generates a random 0-1 matrix indicating whether each criterion is maximization (1) or minimization (0).
    """
    return np.random.randint(0, 2, size=(1, m))  # 1 row, m columns

def normalize_matrix(matrix, criteria_types):
    """
    Normalizes the decision matrix based on whether the criterion is maximized or minimized.
    - If Maximize (1): Normalize by dividing by the max value of the column.
    - If Minimize (0): Normalize by taking the min value of the column divided by the number.
    """
    normalized_matrix = np.zeros_like(matrix, dtype=float)

    for j in range(matrix.shape[1]):  # Loop through each column
        if criteria_types[0, j] == 1:  # Maximization
            normalized_matrix[:, j] = matrix[:, j] / np.max(matrix[:, j])
        else:  # Minimization
            normalized_matrix[:, j] = np.min(matrix[:, j]) / matrix[:, j]

    return normalized_matrix

def compute_wsm_score(normalized_matrix, weights):
    """
    Computes WSM scores by summing the weighted normalized values row-wise.
    """
    weighted_matrix = normalized_matrix * weights  # Element-wise multiplication
    wsm_scores = np.sum(weighted_matrix, axis=1)  # Sum over rows
    return wsm_scores

def compute_wpm_score(normalized_matrix, weights):
    """
    Computes WPM scores by multiplying the normalized values raised to the power of the weights.
    """
    wpm_scores = np.prod(normalized_matrix ** weights, axis=1)  # Multiply each row, with exponentiation by weights
    return wpm_scores

def compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5):
    """
    Computes WASSPASS score as a combination of WSM and WPM.
    """
    wasspass_scores = lambda_value * wsm_scores + (1 - lambda_value) * wpm_scores
    return wasspass_scores

# Function to interact with the user for matrix dimensions
def user_input():
    n = int(input("Enter the number of rows (n): "))  # Number of alternatives
    m = int(input("Enter the number of columns (m): "))  # Number of criteria

    # Generate a random decision matrix (n rows, m columns)
    matrix = np.random.randint(1, 101, size=(n, m))  # Random values between 1 and 100

    # Generate a random 0-1 matrix indicating max/min for each column
    criteria_types = generate_criteria_type_matrix(m)

    print("\nDecision Matrix:")
    print(matrix)

    print("\nCriteria Type Matrix for Random Scenario (0 = Min, 1 = Max):")
    print(criteria_types)

    # Normalize the matrix
    normalized_matrix = normalize_matrix(matrix, criteria_types)

    print("\nNormalized Matrix:")
    print(normalized_matrix)

    return matrix, criteria_types, normalized_matrix

# Get user input
matrix, criteria_types, normalized_matrix = user_input()

# Use previously computed weights (if any for WSM and WPM calculation later)
weights = np.random.rand(matrix.shape[1])  # Random weights for each criterion

# Compute WSM scores
wsm_scores = compute_wsm_score(normalized_matrix, weights)

# Compute WPM scores
wpm_scores = compute_wpm_score(normalized_matrix, weights)

# Compute WASSPASS scores using lambda = 0.5
wasspass_scores = compute_wasspass_score(wsm_scores, wpm_scores, lambda_value=0.5)

# Display the results
print("\nWSM Scores:")
print(wsm_scores)

print("\nWPM Scores:")
print(wpm_scores)

print("\nWASSPASS Scores (Combination of WSM and WPM with λ = 0.5):")
print(wasspass_scores)

Enter the number of rows (n): 5
Enter the number of columns (m): 4

Decision Matrix:
[[19 98 88 28]
 [99 72 17 56]
 [86  5 38 16]
 [ 2 20 70 40]
 [88 59 26  6]]

Criteria Type Matrix for Random Scenario (0 = Min, 1 = Max):
[[0 0 1 1]]

Normalized Matrix:
[[0.10526316 0.05102041 1.         0.5       ]
 [0.02020202 0.06944444 0.19318182 1.        ]
 [0.02325581 1.         0.43181818 0.28571429]
 [1.         0.25       0.79545455 0.71428571]
 [0.02272727 0.08474576 0.29545455 0.10714286]]

WSM Scores:
[1.1143371  0.34906428 0.82336528 1.84828062 0.34167122]

WPM Scores:
[0.03902084 0.00221876 0.01206076 0.46862707 0.00299807]

WASSPASS Scores (Combination of WSM and WPM with λ = 0.5):
[0.57667897 0.17564152 0.41771302 1.15845384 0.17233465]
